In [ ]:
# 1. 돈의 물리학 에 대해 5만권의 코사인 유사도 구하기 

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [38]:
data = pd.read_csv('final_book3_token.csv')
df = pd.DataFrame(data)
df['tokenize_intro']

0        본원 rd 정보 센터 에서는 글로벌 거대 기업 들 약진 거세지다 생 성형 인공 지능...
1        < num > 대박 날 바이오 다크호스 와 함께 미지 길 성공 길 로 < num >...
2        마케팅 으로 세상 바꾸다 최고 브랜드 되다 법 마케팅 전략 과 사례 로 배우다 마케...
3        저 성장 인구 감소 시 대 어떻 게 소비자 설득 하고 새롭다 시장 만들다 것 인가 ...
4        특수 물건 부동산 경매 통해 크다 수익 올리다 싶다 독자 들 위해 다양하다 성공사례...
                               ...                        
56927    상품 yes < num > 에서 구성 한 상품 이다 낱개 반품 불가 도서 연봉 말고...
56928    지금 중국 주식 천만원 면 < num > 년 후 강남 아파트 산다 지금 현재 중국 ...
56929    직장인 위 한 조직 생활 교과서 조직 사회 사회인 라면 누구 나 번은 거치다 인생 ...
56930    상품 yes < num > 에서 구성 한 상품 이다 낱개 반품 불가 도서 충전 수업...
56931    트렌드 코리아 < num > 양 해 소비 트렌드 키우다 들다 count sheep ...
Name: tokenize_intro, Length: 56932, dtype: object

In [82]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['tokenize_intro'][:3000])
tfidf_matrix

<3000x19158 sparse matrix of type '<class 'numpy.float64'>'
	with 278585 stored elements in Compressed Sparse Row format>

In [86]:
tfidf_matrix[114].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [94]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def get_mmr_recommendations(book_title, cosine_sim, df, alpha=0.5, num_recommendations=5):
    # 책 제목에 대한 인덱스 가져오기
    idx = df[df['title'] == book_title].index[0]

    # 관련성 점수 (코사인 유사도)
    scores = cosine_sim[idx]

    # MMR을 위한 초기 후보 목록 생성
    recommended_indices = set()
    ranked_indices = np.argsort(scores)[::-1]  # 내림차순 정렬

    for i in ranked_indices:
        if len(recommended_indices) >= num_recommendations:
            break
        
        if i not in recommended_indices:
            # MMR 계산
            relevance_score = scores[i]
            diversity_score = sum(cosine_sim[i][j] for j in recommended_indices) if recommended_indices else 0
            mmr_score = alpha * relevance_score - (1 - alpha) * diversity_score

            # MMR 점수가 높은 항목 선택
            if mmr_score > 0:
                recommended_indices.add(i)

    # 추천 결과 반환
    recommended_books = df.iloc[list(recommended_indices)].copy()
    recommended_books['similarity'] = [scores[i] for i in recommended_indices]

    return recommended_books[['title', 'similarity']]

data = pd.read_csv('final_book3_token.csv')
df = pd.DataFrame(data)


# TF-IDF 행렬 생성
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['intro'])

# 코사인 유사도 행렬 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 추천할 책 제목
book_title = '돈의 물리학'

# 추천 결과 가져오기
recommendations = get_mmr_recommendations(book_title, cosine_sim, df, alpha=0.5, num_recommendations=5)
print(recommendations)


ParserError: Error tokenizing data. C error: out of memory